![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [3]:
# QuantBook Analysis Tool
# For more information see [https://www.quantconnect.com/docs/v2/our-platform/research/getting-started]
from datetime import datetime
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima.model import ARIMA
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
qb = QuantBook()

iShares_symbol = qb.add_equity('IVV').symbol
vanguard_symbol = qb.add_equity('VOO').symbol

print(vanguard_symbol)

start_date = datetime(2010, 9, 9)
end_date = datetime(2025, 10, 12)
history = qb.history([iShares_symbol, vanguard_symbol], start_date, end_date, Resolution.Daily)

# print(history)
def get_rsquared(model_results, stock_daily_returns):
    fitted_values = model_results.fittedvalues
    actual_values = stock_daily_returns.iloc[1:]  # AutoReg drops first observation

    # Calculate R²
    ss_res = np.sum((actual_values - fitted_values)**2)  # Sum of squared residuals
    ss_tot = np.sum((actual_values - actual_values.mean())**2)  # Total sum of squares
    r_squared = 1 - (ss_res / ss_tot)

    return r_squared
closes = history['close'].copy()
print(closes)
closes.loc['IVV'] = (closes.loc['IVV'].values / closes.loc['IVV'][0]) * 100
closes.loc['VOO'] = (closes.loc['VOO'].values / closes.loc['VOO'][0]) * 100
# df.loc['IVV'] = df.loc['IVV'] / divisors['IVV']
# df.loc['VOO'] = df.loc['VOO'] / divisors['VOO']
print(closes)


daily_returns = closes.groupby(level='symbol').pct_change().dropna().copy()
daily_returns = daily_returns*100

# print(daily_returns.loc['VOO'])
# print(daily_returns.loc['IVV'])

# AR(1) model
iShare_model = AutoReg(daily_returns.loc['IVV'].values, lags = 1)
iShare_results = iShare_model.fit()

rsquared = get_rsquared(iShare_results, daily_returns.loc['IVV'])
print(f"R^2 = {rsquared}")
iShare_results.summary()

# Random walk with drift model
iShare_drift_model = ARIMA(daily_returns.loc['IVV'].values, order=(0,1,0), trend = 't')
iShare_drift_results = iShare_drift_model.fit()

vanguard_drift_model = ARIMA(daily_returns.loc['VOO'].values, order=(0,1,0), trend = 't')
vanguard_drift_results = vanguard_drift_model.fit()

print(iShare_drift_results.summary())
print(vanguard_drift_results.summary())
# Dicky-Fuller test
from statsmodels.tsa.stattools import adfuller

IVV_result = adfuller(daily_returns.loc['IVV'].values, None, 'c', 'AIC')
VOO_result = adfuller(daily_returns.loc['VOO'].values, None, 'c', 'AIC')

print(f"DF IVV tao-stat = {IVV_result[0]}")
print(f"DF VOO tao-stat = {VOO_result[0]}")

# Large tao-stats for both tests > reject null that random walk w drift is non-stationary once differenced
# This implies H1: Yt is stationary with mean β1/(1-⍴)
# The first differences of each timeseries are stationary
# Cointegration
from statsmodels.tsa.stattools import coint
# print(len(daily_returns.loc['IVV'].values))
# print(len(daily_returns.loc['VOO'].values))
# print(daily_returns.loc['IVV'])
# print(daily_returns.loc['VOO'])

IVV_hat = iShare_drift_results.fittedvalues.cumsum()  # reconstruct level from differenced fit
VOO_hat = vanguard_drift_results.fittedvalues.cumsum()

cointegration_result = coint(daily_returns.loc['IVV'].values, daily_returns.loc['VOO'].values, 'c')
print(cointegration_result)
# Results state a cointegrated series is possbile
# Estimating the cointegrated series
import statsmodels.api as sm
from statsmodels.tsa.vector_ar.vecm import VECM
from statsmodels.tsa.vector_ar.vecm import coint_johansen

# x_var = sm.add_constant(iShare_drift_model)
# cointegrated_model = sm.OLS(vanguard_drift_model, x_var).fit()
# cointegrated_model
# VECM()

test_data = np.column_stack([daily_returns.loc['IVV'].values, daily_returns.loc['VOO'].values])

coint_result = coint_johansen(test_data, det_order = 0, k_ar_diff = 1)
w1 = coint_result.evec[0, 0]
w2 = coint_result.evec[1, 0]
print(f"w1 = {w1}, w2 = {w2}")

epsilon = w1 * daily_returns.loc['IVV'].values + w2 * daily_returns.loc['VOO'].values

adf_result = adfuller(epsilon)
print(adf_result)
dates = daily_returns.index.get_level_values(1).unique()

epsilon_mean = np.mean(epsilon)
epsilon_var = np.var(epsilon)
epsilon_std = np.sqrt(epsilon_var)

plt.figure(figsize=(12, 6))
plt.plot(dates, epsilon, label='Residual ε_t')
plt.axhline(y=0, color='r', linestyle='--', linewidth=1, alpha=0.7, label='Zero line')
plt.axhline(y=epsilon_mean, color='r', linestyle='--', linewidth=1, alpha=0.7, label=f'Mean = {epsilon_mean:.4f}')
plt.axhline(y=epsilon_mean + 2*epsilon_std, color='orange', linestyle='--', linewidth=1, alpha=0.7, label=f'+2σ = {epsilon_mean + 2*epsilon_std:.4f}')
plt.axhline(y=epsilon_mean - 2*epsilon_std, color='orange', linestyle='--', linewidth=1, alpha=0.7, label=f'-2σ = {epsilon_mean - 2*epsilon_std:.4f}')
plt.title(f'Stationary Linear Combination: ε_t = {w1:.4f}*IVV + {w2:.4f}*VOO')
plt.xlabel('Date')
plt.ylabel('ε_t')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

# Applying the above application to different ETF pairs
def coint_graph(ticker_1, ticker_2, start_date):
    equity_1 = qb.add_equity(ticker_1).symbol
    equity_2 = qb.add_equity(ticker_2).symbol
    end_date = datetime(2025, 10, 12)

    history = qb.history([equity_1, equity_2], start_date, end_date, Resolution.Daily)
    closes = history['close'].copy()
    closes.loc[ticker_1] = (closes.loc[ticker_1].values / closes.loc[ticker_1].iloc[0]) * 100
    closes.loc[ticker_2] = (closes.loc[ticker_2].values / closes.loc[ticker_2].iloc[0]) * 100
    test_data = np.column_stack([closes.loc[ticker_1].values, closes.loc[ticker_2].values])

    coint_result = coint_johansen(test_data, det_order = 0, k_ar_diff = 1)
    w1 = coint_result.evec[0, 0]
    w2 = coint_result.evec[1, 0]
    epsilon = w1 * closes.loc[ticker_1].values + w2 * closes.loc[ticker_2].values

    adf_result = adfuller(epsilon)
    dates = closes.index.get_level_values(1).unique()
    
    # Calculate mean and std
    epsilon_mean = np.mean(epsilon)
    epsilon_var = np.var(epsilon)
    epsilon_std = np.sqrt(epsilon_var)

    print(f"Dicky-Fuller results {adf_result}")
    print(f"Mean = {epsilon_mean}, Variance = {epsilon_var}, Std = {epsilon_std}")
    
    plt.figure(figsize=(12, 6))
    plt.plot(dates, epsilon)
    plt.axhline(y=0, color='r', linestyle='--', linewidth=1, alpha=0.7, label='Zero line')
    plt.axhline(y=epsilon_mean, color='r', linestyle='--', linewidth=1, alpha=0.7, label=f'Mean = {epsilon_mean:.4f}')
    plt.axhline(y=epsilon_mean + 2*epsilon_std, color='orange', linestyle='--', linewidth=1, alpha=0.7, label=f'+2σ = {epsilon_mean + 2*epsilon_std:.4f}')
    plt.axhline(y=epsilon_mean - 2*epsilon_std, color='orange', linestyle='--', linewidth=1, alpha=0.7, label=f'-2σ = {epsilon_mean - 2*epsilon_std:.4f}')
    plt.title(f'Stationary Linear Combination: ε_t = {w1:.4f}*{ticker_1} + {w2:.4f}*{ticker_2}')
    plt.xlabel('Date')
    plt.ylabel('ε_t')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()


coint_graph("QQQM", "QQQ", datetime(2020, 10, 13))
coint_graph("VOO", "IVV", datetime(2010, 9, 9))
coint_graph("IWM", "VTWO", datetime(2010, 9, 24))
coint_graph("VTI", "IWV", datetime(2001, 6, 1))

coint_graph("IWM", "VTWO", datetime(2022, 9, 24))
coint_graph("IWM", "VTWO", datetime(2023, 9, 24))
coint_graph("IWM", "VTWO", datetime(2024, 9, 24))
    # ETF_A = "VTWO"
    # ETF_B = "IWV"
from dateutil.relativedelta import relativedelta

def iterative_coint_graph(ticker_1, ticker_2, start_date):
    equity_1 = qb.add_equity(ticker_1).symbol
    equity_2 = qb.add_equity(ticker_2).symbol
    end_date = start_date + relativedelta(years=1)

    while end_date <= datetime.now():
        history = qb.history([equity_1, equity_2], start_date, end_date, Resolution.Daily)
        closes = history['close'].copy()
        closes.loc[ticker_1] = (closes.loc[ticker_1].values / closes.loc[ticker_1].iloc[0]) * 100
        closes.loc[ticker_2] = (closes.loc[ticker_2].values / closes.loc[ticker_2].iloc[0]) * 100
        test_data = np.column_stack([closes.loc[ticker_1].values, closes.loc[ticker_2].values])

        coint_result = coint_johansen(test_data, det_order = 0, k_ar_diff = 1)
        w1 = coint_result.evec[0, 0]
        w2 = coint_result.evec[1, 0]
        epsilon = w1 * closes.loc[ticker_1].values + w2 * closes.loc[ticker_2].values

        adf_result = adfuller(epsilon)
        dates = closes.index.get_level_values(1).unique()
        epsilon_var = np.var(epsilon)
        epsilon_std = np.sqrt(epsilon_var)

        print(f"Period: {start_date.date()} to {end_date.date()}")
        print(f"Dicky-Fuller results {adf_result}")
        print(f"Variance = {epsilon_var}, Std = {epsilon_std}")
        print(f"Expected value = {np.mean(epsilon)}\n")
        
        plt.figure(figsize=(12, 6))
        plt.plot(dates, epsilon)
        plt.axhline(y=0, color='r', linestyle='--', linewidth=1, alpha=0.7, label='Zero line')
        plt.axhline(y=np.mean(epsilon), color='r', linestyle='--', linewidth=1, alpha=0.7, label=f'Mean = {np.mean(epsilon):.4f}')
        plt.axhline(y=2*epsilon_std, color='orange', linestyle='--', linewidth=1, alpha=0.7, label=f'+2σ = {2*epsilon_std:.4f}')
        plt.axhline(y=-2*epsilon_std, color='orange', linestyle='--', linewidth=1, alpha=0.7, label=f'-2σ = {-2*epsilon_std:.4f}')
        plt.title(f'{ticker_1}/{ticker_2} ({start_date.date()} to {end_date.date()}): ε_t = {w1:.4f}*{ticker_1} + {w2:.4f}*{ticker_2}')
        plt.xlabel('Date')
        plt.ylabel('ε_t')
        plt.xticks(rotation=45)
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.tight_layout()
        plt.show()

        # Move to next period
        start_date = end_date
        end_date = start_date + relativedelta(years=1)

iterative_coint_graph("IWM", "VTWO", datetime(2020, 9, 24))

# =============== Rolling Window Cointegration Analysis ===============
def rolling_coint_analysis(ticker_1, ticker_2, full_start_date, full_end_date, window_years=2):
    """
    Perform rolling window cointegration analysis.
    
    Parameters:
    - ticker_1, ticker_2: ETF tickers to analyze
    - full_start_date: earliest data to fetch
    - full_end_date: latest data to fetch
    - window_years: size of rolling window in years
    """
    equity_1 = qb.add_equity(ticker_1).symbol
    equity_2 = qb.add_equity(ticker_2).symbol
    
    # Fetch all data once
    history = qb.history([equity_1, equity_2], full_start_date, full_end_date, Resolution.Daily)
    closes = history['close'].copy()
    closes.loc[ticker_1] = (closes.loc[ticker_1].values / closes.loc[ticker_1].iloc[0]) * 100
    closes.loc[ticker_2] = (closes.loc[ticker_2].values / closes.loc[ticker_2].iloc[0]) * 100
    
    # Initialize storage for results
    results = []
    
    current_start = full_start_date
    window_delta = relativedelta(years=window_years)
    
    while current_start + window_delta <= full_end_date:
        current_end = current_start + window_delta
        
        # Filter data for current window
        window_closes_1 = closes.loc[ticker_1]
        window_closes_2 = closes.loc[ticker_2]
        
        # Get dates in window
        dates_all = closes.index.get_level_values(1)
        mask = (dates_all >= current_start) & (dates_all <= current_end)
        
        if mask.sum() < 100:  # Skip if too few observations
            current_start = current_start + relativedelta(months=3)
            continue
            
        window_data = np.column_stack([
            closes.loc[ticker_1][mask].values,
            closes.loc[ticker_2][mask].values
        ])
        
        try:
            coint_result = coint_johansen(window_data, det_order=0, k_ar_diff=1)
            w1 = coint_result.evec[0, 0]
            w2 = coint_result.evec[1, 0]
            epsilon = w1 * window_data[:, 0] + w2 * window_data[:, 1]
            
            adf_result = adfuller(epsilon)
            
            results.append({
                'start': current_start,
                'end': current_end,
                'w1': w1,
                'w2': w2,
                'adf_stat': adf_result[0],
                'adf_pval': adf_result[1],
                'epsilon_mean': np.mean(epsilon),
                'epsilon_std': np.std(epsilon)
            })
            
        except Exception as e:
            print(f"Error in window {current_start} to {current_end}: {e}")
        
        # Move window forward by 3 months
        current_start = current_start + relativedelta(months=3)
    
    # Convert to DataFrame
    results_df = pd.DataFrame(results)
    
    # Plot results
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    
    # Plot ADF statistic over time
    axes[0, 0].plot(results_df['start'], results_df['adf_stat'], marker='o')
    axes[0, 0].axhline(y=-2.86, color='r', linestyle='--', label='5% critical value')
    axes[0, 0].set_title('ADF Statistic Over Time')
    axes[0, 0].set_xlabel('Window Start Date')
    axes[0, 0].set_ylabel('ADF Statistic')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Plot hedge ratios over time
    axes[0, 1].plot(results_df['start'], results_df['w1'], marker='o', label=f'{ticker_1} weight')
    axes[0, 1].plot(results_df['start'], results_df['w2'], marker='o', label=f'{ticker_2} weight')
    axes[0, 1].set_title('Cointegration Weights Over Time')
    axes[0, 1].set_xlabel('Window Start Date')
    axes[0, 1].set_ylabel('Weight')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Plot epsilon std over time
    axes[1, 0].plot(results_df['start'], results_df['epsilon_std'], marker='o', color='green')
    axes[1, 0].set_title('Spread Volatility Over Time')
    axes[1, 0].set_xlabel('Window Start Date')
    axes[1, 0].set_ylabel('Std Dev of ε')
    axes[1, 0].grid(True, alpha=0.3)
    
    # Plot p-values over time
    axes[1, 1].plot(results_df['start'], results_df['adf_pval'], marker='o', color='purple')
    axes[1, 1].axhline(y=0.05, color='r', linestyle='--', label='5% significance')
    axes[1, 1].set_title('ADF Test P-values Over Time')
    axes[1, 1].set_xlabel('Window Start Date')
    axes[1, 1].set_ylabel('P-value')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return results_df

# Example usage
# rolling_results = rolling_coint_analysis("VTI", "IWV", datetime(2010, 1, 1), datetime(2025, 10, 12), window_years=2)

# =============== Copula-based Pairs Trading ===============
from scipy import stats
from copulas.bivariate import Gumbel, Clayton, Frank
import scipy.stats as stats

"""Formation Period (first 12 months)"""
# Define the pair
ETF_A = "VTI"
ETF_B = "IWV"

equity_A = qb.add_equity(ETF_A).symbol
equity_B = qb.add_equity(ETF_B).symbol

# Formation period: 12 months for parameter estimation
formation_start = datetime(2023, 1, 1)
formation_end = datetime(2023, 12, 31)

history_formation = qb.history([equity_A, equity_B], formation_start, formation_end, Resolution.Daily)
closes_formation = history_formation['close'].copy()

# Calculate daily returns (percentage)
returns_A = closes_formation.loc[ETF_A].pct_change().dropna() * 100
returns_B = closes_formation.loc[ETF_B].pct_change().dropna() * 100

# Ensure alignment
aligned_returns = pd.concat([returns_A.rename('A'), returns_B.rename('B')], axis=1, join='inner').dropna()
returns_A_aligned = aligned_returns['A'].values
returns_B_aligned = aligned_returns['B'].values

# -------------------------
# 1) Fit marginal distributions (Student's t)
# -------------------------
IWV_df, IWV_loc, IWV_scale = stats.t.fit(returns_A_aligned)
VTI_df, VTI_loc, VTI_scale = stats.t.fit(returns_B_aligned)

print(f"{ETF_A} t-distribution parameters: df={IWV_df:.2f}, loc={IWV_loc:.4f}, scale={IWV_scale:.4f}")
print(f"{ETF_B} t-distribution parameters: df={VTI_df:.2f}, loc={VTI_loc:.4f}, scale={VTI_scale:.4f}")

# -------------------------
# 2) Transform to uniform marginals
# -------------------------
u = stats.t.cdf(returns_A_aligned, IWV_df, loc=IWV_loc, scale=IWV_scale)
v = stats.t.cdf(returns_B_aligned, VTI_df, loc=VTI_loc, scale=VTI_scale)
uv = np.column_stack([u, v])

# -------------------------
# 3) Fit copula (Gumbel in this case)
# -------------------------
fitted_copula = Gumbel()
fitted_copula.fit(uv)

print(f"Fitted Gumbel copula with theta = {fitted_copula.theta:.4f}")

# -------------------------
# 4) Optionally compare multiple copulas
# -------------------------
    # copula_candidates = {
    #     'clayton': Clayton(),
    #     'frank': Frank(),
    #     'gumbel': Gumbel()
    # }

    # copula_lls = {}
    # for name, cop in copula_candidates.items():
    #     try:
    #         cop.fit(uv)                      # fit method for 'copulas' library
    #         # compute log-likelihood: sum log pdf of joint copula density at uv
    #         ll = np.sum(np.log(cop.probability_density(uv) + 1e-15))
    #         copula_lls[name] = {'copula': cop, 'll': ll}
    #         print(f"{name} log-likelihood value = {ll:.2f}")
    #     except Exception as e:
    #         copula_lls[name] = {'copula': None, 'll': -np.inf}
    #         print(f"{name} fit failed: {e}")

    # # -------------------------
    # # 5) Select best copula by log-likelihood
    # # -------------------------
    # best_copula_name = max(copula_lls.items(), key=lambda kv: kv[1]['ll'])[0]
    # best_copula = copula_lls[best_copula_name]['copula']
    # print("Best copula:", best_copula_name)

    # fitted_copula = Gumbel()
    # fitted_copula.fit(uv)

"""Trading Period (current 12 months)"""
# (1) Calcualte the daily returns of the ETFs during the day
    # -> 
    # new_IWV_returns = [0.1]
    # new_VTI_returns = [1.11]

# (2) Calculate MIt X given Y and MIt Y given X
    # ->
    # u_new = stats.t.cdf(new_IWV_returns, IWV_df, loc=IWV_loc, scale=IWV_scale)
    # v_new = stats.t.cdf(new_VTI_returns, VTI_df, loc=VTI_loc, scale=VTI_scale)
    # uv_new = np.column_stack([u_new, v_new])

    # # # Compute partial derivatives using the built-in method
    # partials_1 = fitted_copula.partial_derivative(uv_new)
    # uv_swapped = uv[:, [1, 0]]
    # partials_2 = fitted_copula.partial_derivative(uv_swapped)

    # FlagX_t = fitted_copula.partial_derivative(uv_new)            # MI(Y|X)
    # uv_swapped = uv_new[:, [1, 0]]
    # FlagY_t = fitted_copula.partial_derivative(uv_swapped)    # MI(X|Y)

    # # FlagX += FlagX_t[t] - 0.5  # for day t
    # # FlagY += FlagY_t[t] - 0.5
# (3) Use these signals to decided on trading or not
    # ->
    # D = 2, S = 0.6
    # When FlagX reaches D, we short-sell stock X and buy stock Y in equal amounts.
    # When FlagX reaches -D, we short-sell stock Y and buy stock X in equal amounts.
    # When FlagY reaches D, we short-sell stock Y and buy stock X in equal amounts.
    # When FlagY reaches -D, we short-sell stock X and buy stock Y in equal amounts.

    # If trades are opened based on FlagX, then they are closed if FlagX returns to zero or
    # reaches stop-loss position S or -S. If they are opened based on FlagY, then they are closed if
    # FlagY returns to zero or reaches stop-loss position S or -S. After trades are closed, both FlagX
    # and FlagY are reset to zero, and all opening trades are closed at the end of the trading period
    # regardless of the values of FlagX and FlagY
new_IWV_returns = [0.222]
new_VTI_returns = [.11]

u_new = stats.t.cdf(new_IWV_returns, IWV_df, loc=IWV_loc, scale=IWV_scale)
v_new = stats.t.cdf(new_VTI_returns, VTI_df, loc=VTI_loc, scale=VTI_scale)
uv_new = np.column_stack([u_new, v_new])

copula_density = fitted_copula.probability_density(uv_new)
# print(copula_density)
theta = fitted_copula.theta


# # Compute partial derivatives using the built-in method
partials_1 = fitted_copula.partial_derivative(uv_new)
uv_swapped = uv_new[:, [1, 0]]
partials_2 = fitted_copula.partial_derivative(uv_swapped)

# partials[:, 0] = ∂C(u,v)/∂u
# partials[:, 1] = ∂C(u,v)/∂v
# print(partials)

print("Partial derivatives for first observation:")
print(f"∂C/∂u: {partials_1[0]}")
print(f"∂C/∂v: {partials_2[0]}")

# ---------- styling helpers ----------
def _nice_ax(ax):
    ax.grid(True, alpha=0.25, linewidth=0.8)
    for side in ("top", "right"):
        ax.spines[side].set_visible(False)

def _tight_show(fig):
    fig.tight_layout()
    plt.show()

# ---------- core charts ----------
def plot_prices_indexed(s1, s2, label1, label2, title_suffix=""):
    """
    s1, s2: pandas Series indexed by dates (already indexed-to-100).
    This function auto-aligns by the intersection of their indexes.
    """
    common_idx = s1.index.intersection(s2.index)
    s1a, s2a = s1.reindex(common_idx), s2.reindex(common_idx)

    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(s1a.index, s1a.values, label=f"{label1} (indexed to 100)")
    ax.plot(s2a.index, s2a.values, label=f"{label2} (indexed to 100)")
    ax.set_title(f"{label1} vs {label2} — Price Levels Indexed to 100 at Start{title_suffix}")
    ax.set_xlabel("Date")
    ax.set_ylabel("Indexed Price Level")
    ax.legend(loc="best")
    _nice_ax(ax); _tight_show(fig)

def plot_stationary_combination(
    dates, epsilon, *, w1=None, w2=None, a_label=None, b_label=None,
    adf_tau=None, adf_p=None, bands="2sigma", annotate=True
):
    mu = float(np.mean(epsilon))
    sig = float(np.std(epsilon, ddof=1))
    k = 2 if bands == "2sigma" else 3

    title_bits = ["Stationary Linear Combination: ε_t"]
    if w1 is not None and w2 is not None and a_label and b_label:
        title_bits.append(f" = {w1:.4f}·{a_label} + {w2:.4f}·{b_label}")
    if adf_tau is not None and adf_p is not None:
        title_bits.append(f"(ADF τ={adf_tau:.3f}, p={adf_p:.3f})")
    title = " — ".join(title_bits)

    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(dates, epsilon, label="Residual ε_t", linewidth=1.5)
    ax.axhline(0.0, linestyle="--", linewidth=1, alpha=0.7, color="r", label="Zero line")
    ax.axhline(mu, linestyle="--", linewidth=1, alpha=0.7, color="r", label=f"Mean(ε_t) = {mu:.4f}")
    ax.axhline(mu + k*sig, linestyle="--", linewidth=1, alpha=0.7, color="orange", label=f"Upper band (+{k}σ) = {mu + k*sig:.4f}")
    ax.axhline(mu - k*sig, linestyle="--", linewidth=1, alpha=0.7, color="orange", label=f"Lower band (−{k}σ) = {mu - k*sig:.4f}")

    ax.set_title(title)
    ax.set_xlabel("Date")
    ax.set_ylabel("Residual ε_t")

    if annotate:
        ax.text(
            0.01, 0.02,
            f"μ={mu:.4f}, σ={sig:.4f}",
            transform=ax.transAxes,
            fontsize=10,
            bbox=dict(facecolor="white", edgecolor="none", alpha=0.7)
        )

    ax.legend(loc="best")
    _nice_ax(ax); _tight_show(fig)

# ---------- convenience wrapper ----------
def show_pair_price_and_epsilon(closes, ticker_a, ticker_b, epsilon, w1, w2, adf_result, eps_index):
    # price chart (uses each series' own index)
    sA = closes.loc[ticker_a]
    sB = closes.loc[ticker_b]
    plot_prices_indexed(sA, sB, ticker_a, ticker_b)

    # residual chart (uses the exact index that matches epsilon)
    plot_stationary_combination(
        dates=eps_index,
        epsilon=epsilon,
        w1=w1, w2=w2,
        a_label=ticker_a, b_label=ticker_b,
        adf_tau=adf_result[0], adf_p=adf_result[1],
        bands="2sigma", annotate=True
    )

# ================= HOW TO CALL (IVV/VOO example) =================
# Build ε on ALIGNED returns so lengths match perfectly:
ret_ivv = daily_returns.loc['IVV']
ret_voo = daily_returns.loc['VOO']
aligned = pd.concat([ret_ivv.rename('IVV'), ret_voo.rename('VOO')], axis=1, join='inner').dropna()

# If you computed w1, w2 from Johansen on (the same) returns, do:
epsilon = w1 * aligned['IVV'].values + w2 * aligned['VOO'].values
eps_index = aligned.index  # <-- correct dates for epsilon

# Fixed: changed adf_eps to adf_result (using the variable computed earlier)
show_pair_price_and_epsilon(closes, 'IVV', 'VOO', epsilon, w1, w2, adf_result, eps_index)

# ================= INSIDE coint_graph / iterative_coint_graph =================
# After you create 'closes' (indexed to 100) and compute w1, w2:
# ret_a = closes.loc[ticker_1].pct_change().dropna()
# ret_b = closes.loc[ticker_2].pct_change().dropna()
# aligned = pd.concat([ret_a.rename(ticker_1), ret_b.rename(ticker_2)], axis=1, join='inner').dropna()
# epsilon = w1*aligned[ticker_1].values + w2*aligned[ticker_2].values
# adf_eps = adfuller(epsilon)
# show_pair_price_and_epsilon(closes, ticker_1, ticker_2, epsilon, w1, w2, adf_eps, aligned.index)